In [ ]:
!pip install openai
import openai
openai.api_key = "OpenAIAPI金鑰"    # 金鑰

!pip install gradio
import gradio as gr

!pip install googlesearch-python
from googlesearch import search
import json                  # 匯入 JSON 模組

In [ ]:
GPT_MODEL = "gpt-3.5-turbo"  # 定義 GPT 模型
SAVE_TALK = 2                # 保留最後2組提問和回覆記錄
dialogue = []                # 對話記錄串列

In [ ]:
def get_response(dialogue):
  response = openai.chat.completions.create(
      model = GPT_MODEL,
      messages = dialogue,
  )
  return response.choices[0].message.content

In [ ]:
def get_webRes(query):
  results = search(query, advanced=True)
  googleAns = next(results)
  return googleAns.description

In [ ]:
query_str = '''
你確實知道以下這件事嗎？

{}

如果不知道, 請以下列JSON格式回答
{{
  "known":"N",
  "keyword":"搜尋關鍵字",
  "ans":""
}}
如果知道, 請以下列JSON格式回答
{{
  "known":"Y",
  "keyword":"",
  "ans":"你的回答"
}}
'''

In [ ]:
def chat(query):
  global dialogue
  response = get_response(
      dialogue+[{
        'role':'user',
        'content':query_str.format(query)}])
  answer = json.loads(response)
  dialogue.append({"role":"user","content":query})
  if answer["known"] == "Y":
    dialogue.append({"role":"assistant",
            "content":answer["ans"]})
    results = f"GPT回答：{answer['ans']}"
  else:
    results = get_webRes(answer["keyword"])
    dialogue.append({"role":"assistant",
            "content":answer["keyword"]})
    results = f"Google搜尋：{results}"
  while len(dialogue) > SAVE_TALK * 2:           # 當提問和回覆記錄超過時
    dialogue.pop(0)                              # 把前面的串列元素移除掉

  return results

In [ ]:
gr.Interface(
    fn = chat,
    inputs = gr.Textbox(label="輸入您的問題："),
    outputs = gr.Textbox(label="AI 的回答："),
    title = "AI 聊天室",
    allow_flagging = "never"                        # 取消 flag 按鈕
).queue().launch()